<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/[NOTEBOOK_ID])

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.

[Start Your Journey](https://www.buildfastwithai.com/genai-course)

---

# Gemini 3 Pro - Alternative Agent Framework (Agno)

**Created by:** @BuildFastWithAI  
**Model:** Google Gemini 3 Pro  
**Last Updated:** November 2025

Build intelligent agents with alternative frameworks.

In [ ]:
!pip install -q google-generativeai langchain langchain-google-genai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 1. Simple Agent with LangChain

In [ ]:
# Define tools
@tool
def get_current_time(format: str = "12h") -> str:
    """Get current time in specified format."""
    from datetime import datetime
    now = datetime.now()
    if format == "24h":
        return now.strftime("%H:%M:%S")
    return now.strftime("%I:%M:%S %p")

@tool
def calculate(expression: str) -> str:
    """Evaluate mathematical expressions."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-3-pro", temperature=0)

# Create agent
tools = [get_current_time, calculate]
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)

# Execute
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3
)

result = agent_executor.invoke({
    "input": "What time is it and what is 25 * 48?"
})
print(result["output"])

## 2. Custom Agent Framework

In [ ]:
from typing import List, Dict, Callable
import json

class SimpleAgent:
    def __init__(self, name: str, llm, tools: List[Callable]):
        self.name = name
        self.llm = llm
        self.tools = {tool.__name__: tool for tool in tools}
        self.memory = []
    
    def run(self, task: str, max_steps: int = 5) -> str:
        """Execute task with ReAct pattern."""
        self.memory.append({"role": "user", "content": task})
        
        for step in range(max_steps):
            # Think
            prompt = self._build_prompt(task)
            response = self.llm.predict(prompt)
            
            print(f"\nStep {step + 1}: {response[:200]}...")
            
            # Check if task is complete
            if "FINAL ANSWER:" in response:
                answer = response.split("FINAL ANSWER:")[1].strip()
                return answer
            
            # Parse tool call
            if "ACTION:" in response and "ACTION INPUT:" in response:
                action = response.split("ACTION:")[1].split("ACTION INPUT:")[0].strip()
                action_input = response.split("ACTION INPUT:")[1].strip()
                
                # Execute tool
                if action in self.tools:
                    result = self.tools[action](action_input)
                    self.memory.append({
                        "role": "tool",
                        "content": f"Tool {action} returned: {result}"
                    })
                    print(f"Tool result: {result}")
        
        return "Task incomplete"
    
    def _build_prompt(self, task: str) -> str:
        tools_desc = "\n".join([
            f"- {name}: {tool.__doc__}"
            for name, tool in self.tools.items()
        ])
        
        return f"""
You are an AI agent. Complete this task: {task}

Available tools:
{tools_desc}

Use this format:
THOUGHT: [your reasoning]
ACTION: [tool name]
ACTION INPUT: [input for tool]

Or if you have the answer:
FINAL ANSWER: [your answer]

Memory: {json.dumps(self.memory[-3:])}
"""

# Test custom agent
def search_web(query: str) -> str:
    """Search the web."""
    return f"Search results for '{query}': AI is transforming industries."

agent = SimpleAgent(
    name="Assistant",
    llm=llm,
    tools=[search_web, calculate]
)

result = agent.run("Search for AI trends and calculate 100 * 25")
print(f"\nFinal: {result}")

## 3. Multi-Agent System

In [ ]:
class AgentOrchestrator:
    def __init__(self, llm):
        self.llm = llm
        self.agents = {}
    
    def register_agent(self, name: str, agent: SimpleAgent):
        """Register an agent."""
        self.agents[name] = agent
    
    def delegate_task(self, task: str) -> str:
        """Delegate task to appropriate agent."""
        # Determine which agent to use
        agent_list = "\n".join([
            f"- {name}: {agent.name}"
            for name, agent in self.agents.items()
        ])
        
        prompt = f"""
Which agent should handle this task?

Task: {task}

Available agents:
{agent_list}

Return only the agent name.
"""
        
        agent_name = self.llm.predict(prompt).strip()
        
        if agent_name in self.agents:
            print(f"Delegating to: {agent_name}")
            return self.agents[agent_name].run(task)
        
        return "No suitable agent found"

# Create specialized agents
research_agent = SimpleAgent("Researcher", llm, [search_web])
math_agent = SimpleAgent("Calculator", llm, [calculate])

# Create orchestrator
orchestrator = AgentOrchestrator(llm)
orchestrator.register_agent("researcher", research_agent)
orchestrator.register_agent("calculator", math_agent)

# Test
result = orchestrator.delegate_task("Calculate 456 * 789")
print(f"\nResult: {result}")

## Key Takeaways

✅ **Agent Patterns:**
- ReAct (Reasoning + Acting)
- Tool use and execution
- Multi-agent orchestration

📌 **Comparison:**
- **CrewAI**: High-level, role-based
- **Custom**: Full control, flexible
- **LangChain**: Rich ecosystem

🔗 **Resources:**
- Follow [@BuildFastWithAI](https://twitter.com/BuildFastWithAI)